In [ ]:
import pandas as pd
from datasets import load_dataset
from tokenizers.implementations import BertWordPieceTokenizer
ds = load_dataset("stanfordnlp/imdb")


# for item in ds["train"]:

ds["train"].to_csv("./train.csv", index=False)

df = pd.read_csv("./train.csv")

reviews = df.to_string()

with open("corpus.txt","w") as f:
    f.writelines(reviews)

# 初始化一个新的 BertWordPieceTokenizer
bert_tokenizer = BertWordPieceTokenizer()

bert_tokenizer.train("./corpus.txt")

bert_tokenizer.get_vocab()

bert_tokenizer.save_model("./tokenizer")

In [ ]:
from tokenizers.implementations import BertWordPieceTokenizer
tokenizer = BertWordPieceTokenizer.from_file("./tokenizer/vocab.txt")

# tokenizer.encode("oh it works just fine").tokens

tokenizer.encode("ohoh i thought it will be works just fine and feichanghao").tokens

In [ ]:
from transformers import BertTokenizerFast,LineByLineTextDataset,DataCollatorForLanguageModeling,TrainingArguments,Trainer
from transformers import BertConfig,BertForMaskedLM
import torch

print(torch.cuda.is_available())

tokenizer = BertTokenizerFast.from_pretrained("./tokenizer")
dataset = LineByLineTextDataset(tokenizer=tokenizer,file_path="./corpus.txt",block_size=128)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=True,mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir="./output",
    logging_steps=10,
    logging_strategy="steps",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    report_to=["wandb"]
)

bert = BertForMaskedLM(BertConfig())

# tiny_bert = BertForMaskedLM(BertConfig(
#     max_position_embeddings=512,
#     num_attention_heads=2,
#     hidden_size=128,
#     num_hidden_layers=2,
#     intermediate_size=521,
# ))

trainer = Trainer(
    model=bert,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

trainer.train()

trainer.save_model("./output/bert")